In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import re
import requests
import csv
import time
import pyarrow.parquet as pq

In [5]:
driver = webdriver.Firefox()

# List pour stocker la data
all_data = []


for page in range(0, 2):
    # Navigate to page
    driver.get(f'https://www.wineandco.com/vins#page_type=taxon&category_id=2&pageNumber={page}')
    time.sleep(1)

    # Handle cookie dialog
    try:
        cookie_button = driver.find_element(By.CLASS_NAME, 'm-modal__close')
        cookie_button.click()
    except:
        # Get product links from the page
        product_links = [x.get_attribute('href') for x in driver.find_elements(By.XPATH,('//div[@class="c-wine"]/a'))]

        # Loop through product links
        for link in product_links:
            # Navigate to product page
            driver.get(link)
            time.sleep(1)

            # Extract product information
            product = driver.find_elements(By.CLASS_NAME,"l-product")

            # Helper function to extract data
            def extract_data(xpath):
                try:
                    data = product.find_element(By.XPATH, xpath).get_attribute('textContent')
                except:
                    data = " "
                return data

            # Basic product information
            name = extract_data('//h1[@class="l-product__infos__name"]')
            price = extract_data('//span[@class="price-display"]')
            rating = extract_data('//div[@class="l-product__infos__avist"]/span')
//good
            # Product details in a table
            try:
                print("test 0")
                WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//table[@class="a-keyvalue prodDetTable"]/tbody/tr')))
            except TimeoutException:
                print("TableNotFound")
            else:
                table = driver.find_element(By.XPATH, '//table[@class="a-keyvalue prodDetTable"]')
                tr_elements = table.find_elements(By.XPATH, 'tbody/tr')
    # Dictionary to store product details
                product_details = {}
                for tr in tr_elements:
                    th_elements = tr.find_elements(By.XPATH, 'th[@class="a-color-secondary a-size-base prodDetSectionEntry"]')
                    td_elements = tr.find_elements(By.XPATH, 'td[@class="a-size-base prodDetAttrValue"]')
                    for th, td in zip(th_elements, td_elements):
                        product_details[th.get_attribute('textContent').strip('\n')] = td.get_attribute('textContent').strip('\n')
                        print("test 3")
               
            # Add all product information to the list
            all_data.append({"Name": name, "Price": price, "Rating": rating, **product_details})

# Convert list of dictionaries to a pandas DataFrame
df = pd.DataFrame(all_data)

# Store DataFrame in a Parquet file
df.to_parquet('dfAmazon.parquet')

# Close the webdriver session
driver.quit()

IndexError: list index out of range